In [10]:
import pandas as pd
import numpy as np

train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
        

#data0 = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/test.csv', header=0,usecols=test_cols,parse_dates=["click_time"], dtype=dtypes)
data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train_sample.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)

print('len read:',len(data))


data['hour'] = data["click_time"].dt.hour.astype('uint8')
data['day'] = data["click_time"].dt.day.astype('uint8')

print('unique days in sample:' ,data['day'].unique())
print('record counts in days in sample:', data[['day','channel']].groupby(['day']).count())

len read: 100000
unique days in sample: [7 9 8 6]
record counts in days in sample:      channel
day         
6       5011
7      32393
8      34035
9      28561


In [13]:
# calculate combined cvr to test hist fts generation

print('app device cvr:', data[['app','device', 'is_attributed']].groupby(['app','device']).mean().to_string())

print('app device non attri count:', data[['app','device', 'is_attributed']].groupby(['app','device']).count().to_string())


#attributed_data = data.query('is_attributed == 1')

#print('len of attributed:',len(attributed_data))

app device cvr:             is_attributed
app device               
1   1            0.000000
    2            0.000000
    3543         0.000000
    3866         0.000000
2   1            0.000000
    2            0.000000
    59           0.000000
3   1            0.000227
    2            0.000000
    5            0.000000
    59           0.000000
4   1            0.000000
    2            0.000000
5   1            0.075269
    2            0.000000
6   1            0.000000
    2            0.000000
7   1            0.000000
    2            0.000000
    3543         0.000000
    3866         0.000000
8   1            0.002191
    2            0.000000
    59           0.000000
9   1            0.000839
    2            0.001567
    59           0.000000
    3543         0.000000
    3866         0.000000
10  1            0.046392
11  1            0.001050
    2            0.000000
12  1            0.000080
    2            0.000000
    59           0.000000
    3543         0.000

In [13]:
print(1.0 - 456846/184903890)

0.997529278589001


In [ ]:
print('sampling data')
#data = data.set_index('ip').loc[lambda x: (x.index + 401) % 10 == 0].reset_index()


data.drop('click_time', inplace=True, axis=1)

In [25]:
data0['hour'] = data0["click_time"].dt.hour.astype('uint8')
data0['day'] = data0["click_time"].dt.day.astype('uint8')

data0.drop('click_time', inplace=True, axis=1)

In [15]:
#data.drop('click_time', inplace=True, axis=1)

print('added hour and day')



added hour and day


In [26]:
#print(data.describe())

data0 = data0.query('hour in [{}]'.format(','.join(map(lambda x: str(x), set(data['hour'].unique())))))
print(data0['hour'].unique())

hour_set = set(data['hour'].unique())
hour_set0 = set(data0['hour'].unique())


[14 15  4  5  6  9 10 11 13]


In [27]:
osset = set(data['os'].unique())
osset0 = set(data0['os'].unique())

print(len(osset & osset0))
print(len(osset0))

315
573


In [28]:
chset = set(data['app'].unique())
chset0 = set(data0['app'].unique())

print(len(chset & chset0))
print(len(chset0))

384
610
